<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/aburdenko/gcp-jupyter-notebooks/blob/master/vertex-pipelines/kfp-sklearn.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/aburdenko/gcp-jupyter-notebooks/blob/main/vertex-pipelines/kfp-sklearn.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.sandbox.google.com/github/aburdenko/gcp-jupyter-notebooks/blob/main/vertex-pipelines/kfp-sklearn.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
    <td><a target="_blank"  href="https://console.cloud.google.com/vertex-ai/workbench/list/instances"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/a/a0/Google_Cloud_Workbench.png" /> Run in Google Cloud Vertex Workbench</a></td>
</table>


Based on [Colab Notebooks](https://console.cloud.google.com/marketplace/product/colab-marketplace-image-public/colab) available in GCP Marketplace.

In [1]:
USER = "aburdenko@google.com" # @param {type:"string"} <---CHANGE THESE
BUCKET_NAME = "alphafold_protein_structure" # @param {type:"string"} <---CHANGE THESE
GOOGLE_CLOUD_PROJECT = "aburdenko-project" # @param {type:"string"} <---CHANGE THESE
REGION = "us-central1" # @param {type:"string"} <---CHANGE THESE

PIPELINE_ROOT = "{}/pipeline_root/{}".format(BUCKET_NAME, USER)
PIPELINE_ROOT
GOOGLE_APPLICATION_CREDENTIALS="/home/aburdenko/aburdenko-project-d93f3d235d90.json" # @param {type:"string"} <---CHANGE THESE

DRIVE_PATH="/mnt/gdrive"
LIB_PATH=f"{DRIVE_PATH}/Colab Notebooks/lib"
CLEAN_LIB_PATH='/home/aburdenko/python_lib'

print(f"Google Drive Path is: {DRIVE_PATH}")
print(f"Lib Path is: {LIB_PATH}")

%env GOOGLE_CLOUD_PROJECT=$GOOGLE_CLOUD_PROJECT
%env GOOGLE_APPLICATION_CREDENTIALS=$GOOGLE_APPLICATION_CREDENTIALS

Google Drive Path is: /mnt/gdrive
Lib Path is: /mnt/gdrive/Colab Notebooks/lib
env: GOOGLE_CLOUD_PROJECT=aburdenko-project
env: GOOGLE_APPLICATION_CREDENTIALS=/home/aburdenko/aburdenko-project-d93f3d235d90.json


In [2]:
import os
import sys
import errno

if not os.path.islink(CLEAN_LIB_PATH):  
  import os, sys  
  
  if os.path.exists(CLEAN_LIB_PATH):
    os.unlink(CLEAN_LIB_PATH)
    

  try:
    os.symlink(LIB_PATH, CLEAN_LIB_PATH)
  except OSError as e:    
    if e.errno == errno.EEXIST:        
        pass
    
os.listdir(CLEAN_LIB_PATH)        
sys.path.insert(0,CLEAN_LIB_PATH)

if '.' not in sys.path:
  sys.path.insert(0, '.')

In [3]:
import os
libs = os.listdir(CLEAN_LIB_PATH)

In [4]:
needs_restart=False
if not any('google_cloud_aiplatform' in lib for lib in libs):
  print('google_cloud_aiplatform')
  !pip3 install --target=$CLEAN_LIB_PATH google-cloud-aiplatform 
  needs_restart=True  

if not any('google_cloud_pipeline_components' in lib for lib in libs):
  print('google_cloud_pipeline_components')
  !pip3 install --target=$CLEAN_LIB_PATH google-cloud-pipeline-components 
  needs_restart=True  


if not any('kfp' in lib for lib in libs):
  print('kfp')
  !pip3 install --target=$CLEAN_LIB_PATH kfp 
  needs_restart=True  
  

if needs_restart:
  print("🔁 Restarting kernel...")  
  import IPython
  IPython.Application.instance().kernel.do_shutdown(True)

# Authentication and Authorization

In [5]:
!echo $GOOGLE_CLOUD_PROJECT
!gcloud config set project $GOOGLE_CLOUD_PROJECT

aburdenko-project
Updated property [core/project].


In [ ]:
!gcloud --project $GOOGLE_CLOUD_PROJECT services enable compute.googleapis.com         \
                       containerregistry.googleapis.com  \
                       aiplatform.googleapis.com  \
                       cloudbuild.googleapis.com \
                       cloudfunctions.googleapis.com

In [6]:
from inspect import isdatadescriptor
from typing import NamedTuple

from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component, OutputPath, InputPath, Output, Input, Dataset, Metrics, Model, Artifact
from kfp.v2.google.client import AIPlatformClient
#from google.cloud import automl
from google_cloud_pipeline_components import aiplatform as gcc_aip

ImportError: cannot import name 'cygrpc' from 'grpc._cython' (/home/aburdenko/python_lib/grpc/_cython/__init__.py)

In [ ]:
import sys
print(sys.path)

['/mnt/gdrive/notebooks/lib', '/mnt/gdrive/workspace/alphafold-inference-pipeline/notebook', '/mnt/gdrive/workspace/alphafold-inference-pipeline/notebook/$PYTHONPATH', '/mnt/gdrive/workspace/alphafold-inference-pipeline', '/mnt/gdrive/workspace/alphafold-inference-pipeline/notebook', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/aburdenko/.local/lib/python3.9/site-packages', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages']


# End of Setup - Get to Work

In [6]:
@component(
    packages_to_install=["google-cloud-bigquery", "pandas", "pyarrow"],
    base_image="python:3.9",
    output_component_file="create_dataset.yaml"
)
def get_dataframes(
    bq_table: str,    
    output_train_dataset: OutputPath("Dataset"),
    output_eval_dataset: OutputPath("Dataset")
):
    from google.cloud import bigquery
    import pandas as pd

    bqclient = bigquery.Client()
    table = bigquery.TableReference.from_string(
        bq_table
    )
    rows = bqclient.list_rows(
        table
    )
    dataframe = rows.to_dataframe(
        create_bqstorage_client=True,
    )
    dataframe = dataframe.sample(frac=1, random_state=2)    
    dataframe.to_csv(output_train_dataset)

    dataframe = dataframe.sample(frac=1, random_state=2)
    dataframe.to_csv(output_eval_dataset)

In [2]:

import os

PIPELINE_NAME = 'alphafold-inference'
PIPELINE_DESCRIPTION = 'Alphafold inference'

REFERENCE_DATASETS_IMAGE = "https://www.googleapis.com/compute/v1/projects/jk-mlops-dev/global/images/jk-alphafold-datasets 3000"
REFERENCE_DATASETS_GCS_LOCATION = 'gs://alphafold_protein_structure'
REFERENCE_DATASETS_URI = '10.71.1.10,/datasets_v1,/mnt/nfs/alphafold,projects/895222332033/global/networks/default'
MODEL_PARAMS_GCS_LOCATION='gs://alphafold_protein_structure/upload/model_params'
MODEL_PARAMS_URI='gs://alphafold_protein_structure'

UNIREF90_PATH = 'uniref90/uniref90.fasta'
MGNIFY_PATH = 'mgnify/mgy_clusters_2018_12.fa'
BFD_PATH = 'bfd/bfd_metaclust_clu_complete_id30_c90_final_seq.sorted_opt'
UNICLUST30_PATH = 'uniclust30/uniclust30_2018_08/uniclust30_2018_08'
UNIPROT_PATH = 'uniprot/uniprot.fasta'
PDB70_PATH = 'pdb70/pdb70'
PDB_MMCIF_PATH = 'pdb_mmcif/mmcif_files'
PDB_OBSOLETE_PATH = 'pdb_mmcif/obsolete.dat'
PDB_SEQRES_PATH = 'pdb_seqres/pdb_seqres.txt'

UNIREF90 = 'uniref90'
MGNIFY = 'mgnify'
BFD = 'bfd'
UNICLUST30 = 'uniclust30'
PDB70 = 'pdb70'
PDB_MMCIF = 'pdb_mmcif'
PDB_OBSOLETE = 'pdb_obsolete'
PDB_SEQRES = 'pdb_seqres'
UNIPROT = 'uniprot'

RELAX_MEMORY_LIMIT = os.getenv("MEMORY_LIMIT", "85")
RELAX_CPU_LIMIT = os.getenv("CPU_LIMIT", "12")
RELAX_GPU_LIMIT = os.getenv("GPU_LIMIT", "1")
RELAX_GPU_TYPE = os.getenv("GPU_TYPE", "nvidia-tesla-a100")

MEMORY_LIMIT = os.getenv("MEMORY_LIMIT", "85")
CPU_LIMIT = os.getenv("CPU_LIMIT", "12")
GPU_LIMIT = os.getenv("GPU_LIMIT", "1")
GPU_TYPE = os.getenv("GPU_TYPE", "nvidia-tesla-a100")
GKE_ACCELERATOR_KEY = 'cloud.google.com/gke-accelerator'

ALPHAFOLD_COMPONENTS_IMAGE = os.getenv("ALPHAFOLD_COMPONETS_IMAGE", 'gcr.io/aburdenko-project/alphafold')

XLA_PYTHON_CLIENT_MEM_FRACTION = "4.0"
TF_FORCE_UNIFIED_MEMORY = "1"
FASTA_PATH='gs://alphafold_protein_structure/upload/sequences.fasta'
SEQUENCE_DESC='T1050 A7LXT1, Bacteroides Ovatus, 779 residues'
MAX_TEMPLATE_DATE='2020-05-14'
NUM_ENSUMBLE=1
USE_GPU_FOR_RELAXATION=True

@dsl.pipeline(
    name="sklearn-demo",
    description="A simple sklearn demo",
    pipeline_root=PIPELINE_ROOT,
)
def pipeline(    
    project : str = GOOGLE_CLOUD_PROJECT
    , region: str =  REGION
    , sequence_path: str = ""
    , model_params_uri: str=MODEL_PARAMS_GCS_LOCATION
):    
    input_sequence = dsl.importer(
        artifact_uri=sequence_path,
        artifact_class=Dataset,
        reimport=True)
    input_sequence.set_display_name('Input sequence')

    model_parameters = dsl.importer(
        artifact_uri=model_params_uri,
        artifact_class=Artifact,
        reimport=True)
    model_parameters.set_display_name('Model parameters')

    reference_databases = dsl.importer(
        artifact_uri=REFERENCE_DATASETS_URI,
        artifact_class=Dataset,
        reimport=False,
        metadata={
            UNIREF90: UNIREF90_PATH,
            MGNIFY: MGNIFY_PATH,
            BFD: BFD_PATH,
            UNICLUST30: UNICLUST30_PATH,
            PDB70: PDB70_PATH,
            PDB_MMCIF: PDB_MMCIF_PATH,
            PDB_OBSOLETE: PDB_OBSOLETE_PATH,
            PDB_SEQRES: PDB_SEQRES_PATH,
            UNIPROT: UNIPROT_PATH,
            }    
    )
    reference_databases.set_display_name('Reference databases')

    search_uniref = JackhmmerOp(
        project=project,
        region=region,
        database=config.UNIREF90,
        reference_databases=reference_databases.output,
        sequence=input_sequence.output,
    )
    search_uniref.set_display_name('Search Uniref')#.set_caching_options(enable_caching=True)

    search_mgnify = JackhmmerOp(
        project=project,
        region=region,
        database=config.MGNIFY,
        reference_databases=reference_databases.output,
        sequence=input_sequence.output,
    )
    search_mgnify.set_display_name('Search Mgnify')#.set_caching_options(enable_caching=True)

    search_uniclust = HHBlitsOp(
        project=project,
        region=region,
        msa_dbs=[config.UNICLUST30],
        reference_databases=reference_databases.output,
        sequence=input_sequence.output,
    )
    search_uniclust.set_display_name('Search Uniclust')#.set_caching_options(enable_caching=True)

    search_bfd = HHBlitsOp(
        project=project,
        region=region,
        msa_dbs=[config.BFD],
        reference_databases=reference_databases.output,
        sequence=input_sequence.output,
    )
    search_bfd.set_display_name('Search BFD')#.set_caching_options(enable_caching=True)

    search_pdb = HHSearchOp(
        project=project,
        region=region,
        template_dbs=[config.PDB70],
        mmcif_db=config.PDB_MMCIF,
        obsolete_db=config.PDB_OBSOLETE,
        max_template_date=MAX_TEMPLATE_DATE,
        reference_databases=reference_databases.output,
        sequence=input_sequence.output,
        msa=search_uniref.outputs['msa'],
    )
    search_pdb.set_display_name('Search Pdb')#.set_caching_options(enable_caching=True)

    aggregate_features = AggregateFeaturesOp(
        sequence=input_sequence.output,
        msa1=search_uniref.outputs['msa'],
        msa2=search_mgnify.outputs['msa'],
        msa3=search_bfd.outputs['msa'],
        msa4=search_uniclust.outputs['msa'],
        template_features=search_pdb.outputs['template_features'],
    )
    aggregate_features.set_display_name('Aggregate features')#.set_caching_options(enable_caching=True)

    # Think what to do with random seed when switch to Parallel loop
    with dsl.ParallelFor(models) as model:
        model_predict = ModelPredictOp(
            model_features=aggregate_features.outputs['features'],
            model_params=model_parameters.output,
            model_name=model.model_name,
            num_ensemble=NUM_ENSUMBLE,
            random_seed=model.random_seed
        )
        model_predict.set_display_name('Predict')#.set_caching_options(enable_caching=True)
        model_predict.set_cpu_limit(config.CPU_LIMIT)
        model_predict.set_memory_limit(config.MEMORY_LIMIT)
        model_predict.set_gpu_limit(config.GPU_LIMIT)
        model_predict.add_node_selector_constraint(config.GKE_ACCELERATOR_KEY, config.GPU_TYPE)
        model_predict.set_env_variable("TF_FORCE_UNIFIED_MEMORY", config.TF_FORCE_UNIFIED_MEMORY)
        model_predict.set_env_variable("XLA_PYTHON_CLIENT_MEM_FRACTION", config.XLA_PYTHON_CLIENT_MEM_FRACTION)

        relax_protein = RelaxProteinOp(
            unrelaxed_protein=model_predict.outputs['unrelaxed_protein'],
            use_gpu=USE_GPU_FOR_RELAXATION
        )
        relax_protein.set_display_name('Relax protein')#.set_caching_options(enable_caching=True)
        relax_protein.set_cpu_limit(config.RELAX_CPU_LIMIT)
        relax_protein.set_memory_limit(config.RELAX_MEMORY_LIMIT)
        relax_protein.set_gpu_limit(config.RELAX_GPU_LIMIT)
        relax_protein.add_node_selector_constraint(config.GKE_ACCELERATOR_KEY, config.RELAX_GPU_TYPE)
        relax_protein.set_env_variable("TF_FORCE_UNIFIED_MEMORY", config.TF_FORCE_UNIFIED_MEMORY)
        relax_protein.set_env_variable("XLA_PYTHON_CLIENT_MEM_FRACTION", config.XLA_PYTHON_CLIENT_MEM_FRACTION)
    

from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="custom_train_pipeline.json"
)


NameError: name 'dsl' is not defined

In [17]:
from datetime import datetime
# from google.cloud import aiplatform
# from google.cloud.aiplatform import pipeline_jobs


api_client = AIPlatformClient(
    project_id=GOOGLE_CLOUD_PROJECT,
    region=REGION,
)

PIPELINE_SPEC_PATH="custom_train_pipeline.json"
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(BUCKET_NAME, USER)

params = {
    'sequence_path': FASTA_PATH
    , 'model_params_uri': MODEL_PARAMS_GCS_LOCATION
    # 'sequence_desc': SEQUENCE_DESC,
    # 'max_template_date': MAX_TEMPLATE_DATE,
    # 'project': GOOGLE_CLOUD_PROJECT,
    # 'region': REGION,
    # 'models': models,
    # 'num_ensemble': NUM_ENSUMBLE,
    # 'use_gpu_for_relaxation': USE_GPU_FOR_RELAXATION
}

compiler.Compiler().compile(
   pipeline_func=pipeline, package_path=PIPELINE_SPEC_PATH
)
api_client = AIPlatformClient(project_id=GOOGLE_CLOUD_PROJECT, region=REGION)
response = api_client.create_run_from_job_spec(
 PIPELINE_SPEC_PATH,
 pipeline_root=PIPELINE_ROOT,
 parameter_values=params
)

In [49]:
import config
from typing import List
from typing import Mapping
from google.cloud import aiplatform

from alphafold_components import  (
    RelaxProteinOp, AggregateFeaturesOp, ModelPredictOp, 
    JackhmmerOp, HHBlitsOp, HHSearchOp, ImportSeqenceOp
)
    
@dsl.pipeline(name=config.PIPELINE_NAME, description=config.PIPELINE_DESCRIPTION)
def pipeline(

    sequence_path: str
    # sequence_desc: str,
    # project: str='aburdenk-project',
    # region: str='us-central1',
    # max_template_date: str='2020-05-14',
    # models: List[Mapping]=[{'model_name': 'model_1', 'random_seed': 1}],
    # use_gpu_for_relaxation: bool=True,
    # num_ensemble: int=1,
    # reference_datasets_uri: str=config.REFERENCE_DATASETS_URI, 
    # model_params_uri: str=config.MODEL_PARAMS_GCS_LOCATION
):
    """Runs AlphaFold inference."""


    input_sequence = dsl.importer(
        artifact_uri=sequence_path,
        artifact_class=Dataset,
        reimport=True)
    input_sequence.set_display_name('Input sequence')

    model_parameters = dsl.importer(
        artifact_uri=model_params_uri,
        artifact_class=Artifact,
        reimport=True)
    model_parameters.set_display_name('Model parameters')

    reference_databases = dsl.importer(
        artifact_uri=reference_datasets_uri,
        artifact_class=Dataset,
        reimport=False,
        metadata={
            config.UNIREF90: config.UNIREF90_PATH,
            config.MGNIFY: config.MGNIFY_PATH,
            config.BFD: config.BFD_PATH,
            config.UNICLUST30: config.UNICLUST30_PATH,
            config.PDB70: config.PDB70_PATH,
            config.PDB_MMCIF: config.PDB_MMCIF_PATH,
            config.PDB_OBSOLETE: config.PDB_OBSOLETE_PATH,
            config.PDB_SEQRES: config.PDB_SEQRES_PATH,
            config.UNIPROT: config.UNIPROT_PATH,
            }    
    )
    reference_databases.set_display_name('Reference databases')


    # search_uniref = JackhmmerOp(
    #     project=project,
    #     region=region,
    #     database=config.UNIREF90,
    #     reference_databases=reference_databases.output,
    #     sequence=input_sequence.output,
    # )
    # search_uniref.set_display_name('Search Uniref')#.set_caching_options(enable_caching=True)

    # search_mgnify = JackhmmerOp(
    #     project=project,
    #     region=region,
    #     database=config.MGNIFY,
    #     reference_databases=reference_databases.output,
    #     sequence=input_sequence.output,
    # )
    # search_mgnify.set_display_name('Search Mgnify')#.set_caching_options(enable_caching=True)

    # search_uniclust = HHBlitsOp(
    #     project=project,
    #     region=region,
    #     msa_dbs=[config.UNICLUST30],
    #     reference_databases=reference_databases.output,
    #     sequence=input_sequence.output,
    # )
    # search_uniclust.set_display_name('Search Uniclust')#.set_caching_options(enable_caching=True)

    # search_bfd = HHBlitsOp(
    #     project=project,
    #     region=region,
    #     msa_dbs=[config.BFD],
    #     reference_databases=reference_databases.output,
    #     sequence=input_sequence.output,
    # )
    # search_bfd.set_display_name('Search BFD')#.set_caching_options(enable_caching=True)

    # search_pdb = HHSearchOp(
    #     project=project,
    #     region=region,
    #     template_dbs=[config.PDB70],
    #     mmcif_db=config.PDB_MMCIF,
    #     obsolete_db=config.PDB_OBSOLETE,
    #     max_template_date=max_template_date,
    #     reference_databases=reference_databases.output,
    #     sequence=input_sequence.output,
    #     msa=search_uniref.outputs['msa'],
    # )
    # search_pdb.set_display_name('Search Pdb')#.set_caching_options(enable_caching=True)

    # aggregate_features = AggregateFeaturesOp(
    #     sequence=input_sequence.output,
    #     msa1=search_uniref.outputs['msa'],
    #     msa2=search_mgnify.outputs['msa'],
    #     msa3=search_bfd.outputs['msa'],
    #     msa4=search_uniclust.outputs['msa'],
    #     template_features=search_pdb.outputs['template_features'],
    # )
    # aggregate_features.set_display_name('Aggregate features')#.set_caching_options(enable_caching=True)

    # # Think what to do with random seed when switch to Parallel loop
    # with dsl.ParallelFor(models) as model:
    #     model_predict = ModelPredictOp(
    #         model_features=aggregate_features.outputs['features'],
    #         model_params=model_parameters.output,
    #         model_name=model.model_name,
    #         num_ensemble=num_ensemble,
    #         random_seed=model.random_seed
    #     )
    #     model_predict.set_display_name('Predict')#.set_caching_options(enable_caching=True)
    #     model_predict.set_cpu_limit(config.CPU_LIMIT)
    #     model_predict.set_memory_limit(config.MEMORY_LIMIT)
    #     model_predict.set_gpu_limit(config.GPU_LIMIT)
    #     model_predict.add_node_selector_constraint(config.GKE_ACCELERATOR_KEY, config.GPU_TYPE)
    #     model_predict.set_env_variable("TF_FORCE_UNIFIED_MEMORY", config.TF_FORCE_UNIFIED_MEMORY)
    #     model_predict.set_env_variable("XLA_PYTHON_CLIENT_MEM_FRACTION", config.XLA_PYTHON_CLIENT_MEM_FRACTION)

    #     relax_protein = RelaxProteinOp(
    #         unrelaxed_protein=model_predict.outputs['unrelaxed_protein'],
    #         use_gpu=use_gpu_for_relaxation,
    #     )
    #     relax_protein.set_display_name('Relax protein')#.set_caching_options(enable_caching=True)
    #     relax_protein.set_cpu_limit(config.RELAX_CPU_LIMIT)
    #     relax_protein.set_memory_limit(config.RELAX_MEMORY_LIMIT)
    #     relax_protein.set_gpu_limit(config.RELAX_GPU_LIMIT)
    #     relax_protein.add_node_selector_constraint(config.GKE_ACCELERATOR_KEY, config.RELAX_GPU_TYPE)
    #     relax_protein.set_env_variable("TF_FORCE_UNIFIED_MEMORY", config.TF_FORCE_UNIFIED_MEMORY)
    #     relax_protein.set_env_variable("XLA_PYTHON_CLIENT_MEM_FRACTION", config.XLA_PYTHON_CLIENT_MEM_FRACTION)
    

from kfp.v2 import compiler  

compiler.Compiler().compile(
        pipeline_func=pipeline, package_path=f'{config.PIPELINE_NAME}.json'    
)


In [50]:
@dsl.pipeline(
    name="sklearn-demo",
    description="A simple sklearn demo",
    pipeline_root=PIPELINE_ROOT,
)
def pipeline(
    sequence_path: str = "",
    output_data_path: str = "data.csv",
    project: str = GOOGLE_CLOUD_PROJECT,
    region: str = REGION
):
    input_sequence = dsl.importer(
        artifact_uri=sequence_path,
        artifact_class=Dataset,
        reimport=True)
    
    input_sequence.set_display_name('Input sequence')

from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="custom_train_pipeline.json"
)    

In [51]:
import os

PIPELINE_NAME = 'alphafold-inference'
PIPELINE_DESCRIPTION = 'Alphafold inference'

REFERENCE_DATASETS_IMAGE = "https://www.googleapis.com/compute/v1/projects/jk-mlops-dev/global/images/jk-alphafold-datasets 3000"
REFERENCE_DATASETS_GCS_LOCATION = 'gs://alphafold_protein_structure'
REFERENCE_DATASETS_URI = '10.71.1.10,/datasets_v1,/mnt/nfs/alphafold,projects/895222332033/global/networks/default'
MODEL_PARAMS_GCS_LOCATION='gs://alphafold_protein_structure/upload/model_params'
MODEL_PARAMS_URI='gs://alphafold_protein_structure'

UNIREF90_PATH = 'uniref90/uniref90.fasta'
MGNIFY_PATH = 'mgnify/mgy_clusters_2018_12.fa'
BFD_PATH = 'bfd/bfd_metaclust_clu_complete_id30_c90_final_seq.sorted_opt'
UNICLUST30_PATH = 'uniclust30/uniclust30_2018_08/uniclust30_2018_08'
UNIPROT_PATH = 'uniprot/uniprot.fasta'
PDB70_PATH = 'pdb70/pdb70'
PDB_MMCIF_PATH = 'pdb_mmcif/mmcif_files'
PDB_OBSOLETE_PATH = 'pdb_mmcif/obsolete.dat'
PDB_SEQRES_PATH = 'pdb_seqres/pdb_seqres.txt'

UNIREF90 = 'uniref90'
MGNIFY = 'mgnify'
BFD = 'bfd'
UNICLUST30 = 'uniclust30'
PDB70 = 'pdb70'
PDB_MMCIF = 'pdb_mmcif'
PDB_OBSOLETE = 'pdb_obsolete'
PDB_SEQRES = 'pdb_seqres'
UNIPROT = 'uniprot'

RELAX_MEMORY_LIMIT = os.getenv("MEMORY_LIMIT", "85")
RELAX_CPU_LIMIT = os.getenv("CPU_LIMIT", "12")
RELAX_GPU_LIMIT = os.getenv("GPU_LIMIT", "1")
RELAX_GPU_TYPE = os.getenv("GPU_TYPE", "nvidia-tesla-a100")

MEMORY_LIMIT = os.getenv("MEMORY_LIMIT", "85")
CPU_LIMIT = os.getenv("CPU_LIMIT", "12")
GPU_LIMIT = os.getenv("GPU_LIMIT", "1")
GPU_TYPE = os.getenv("GPU_TYPE", "nvidia-tesla-a100")
GKE_ACCELERATOR_KEY = 'cloud.google.com/gke-accelerator'

ALPHAFOLD_COMPONENTS_IMAGE = os.getenv("ALPHAFOLD_COMPONETS_IMAGE", 'gcr.io/aburdenko-project/alphafold')

XLA_PYTHON_CLIENT_MEM_FRACTION = "4.0"
TF_FORCE_UNIFIED_MEMORY = "1"
FASTA_PATH='gs://alphafold_protein_structure/upload/sequences.fasta'
SEQUENCE_DESC='T1050 A7LXT1, Bacteroides Ovatus, 779 residues'
MAX_TEMPLATE_DATE='2020-05-14'
NUM_ENSUMBLE=1
USE_GPU_FOR_RELAXATION=True



from datetime import datetime
# from google.cloud import aiplatform
# from google.cloud.aiplatform import pipeline_jobs


api_client = AIPlatformClient(
    project_id=GOOGLE_CLOUD_PROJECT,
    region=REGION,
)

PIPELINE_SPEC_PATH="custom_train_pipeline.json"
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(BUCKET_NAME, USER)

params = {
    'sequence_path': FASTA_PATH
    # 'sequence_desc': SEQUENCE_DESC,
    # 'max_template_date': MAX_TEMPLATE_DATE,
    # 'project': GOOGLE_CLOUD_PROJECT,
    # 'region': REGION,
    # 'models': models,
    # 'num_ensemble': NUM_ENSUMBLE,
    # 'use_gpu_for_relaxation': USE_GPU_FOR_RELAXATION
}

compiler.Compiler().compile(
   pipeline_func=pipeline, package_path=PIPELINE_SPEC_PATH
)
api_client = AIPlatformClient(project_id=GOOGLE_CLOUD_PROJECT, region=REGION)
response = api_client.create_run_from_job_spec(
 PIPELINE_SPEC_PATH,
 pipeline_root=PIPELINE_ROOT,
 parameter_values=params
)

TypeError: string indices must be integers

In [41]:
from datetime import datetime
# from google.cloud import aiplatform
# from google.cloud.aiplatform import pipeline_jobs


api_client = AIPlatformClient(
    project_id=GOOGLE_CLOUD_PROJECT,
    region=REGION,
)

PIPELINE_SPEC_PATH="custom_train_pipeline.json"
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(BUCKET_NAME, USER)

compiler.Compiler().compile(
   pipeline_func=pipeline, package_path=PIPELINE_SPEC_PATH
)

models = [
    {
        'model_name': 'model_1', 'random_seed': 1,
    },
#    {
#        'model_name': 'model_2', 'random_seed': 2,
#    },
#    {
#        'model_name': 'model_3', 'random_seed': 3,
#    },
#            {
#        'model_name': 'model_4', 'random_seed': 4,
#    },
#            {
#        'model_name': 'model_5', 'random_seed': 5,
#    },

]

FASTA_PATH='gs://alphafold_protein_structure/upload/sequences.fasta'
SEQUENCE_DESC='T1050 A7LXT1, Bacteroides Ovatus, 779 residues'
MAX_TEMPLATE_DATE='2020-05-14'
NUM_ENSUMBLE=1
USE_GPU_FOR_RELAXATION=True


params = {
    'sequence_path': FASTA_PATH
    # 'sequence_desc': SEQUENCE_DESC,
    # 'max_template_date': MAX_TEMPLATE_DATE,
    # 'project': GOOGLE_CLOUD_PROJECT,
    # 'region': REGION,
    # 'models': models,
    # 'num_ensemble': NUM_ENSUMBLE,
    # 'use_gpu_for_relaxation': USE_GPU_FOR_RELAXATION
}

# pipeline_job = aip.PipelineJob(
#     display_name=_PIPELINE_JOB_NAME,
#     template_path=FLAGS.pipeline_spec,
#     pipeline_root=f'{FLAGS.pipeline_staging_location}/{_PIPELINE_JOB_NAME}',
#     parameter_values=params,
#     enable_caching=FLAGS.enable_caching,
# )

# pipeline_job.run(
#     service_account=FLAGS.pipelines_sa    
# )


# api_client = AIPlatformClient(project_id=GOOGLE_CLOUD_PROJECT, region=REGION)
# response = api_client.create_run_from_job_spec
# (
#     PIPELINE_SPEC_PATH,
#     pipeline_root=PIPELINE_ROOT,
#     parameter_values=params
# )

# sequence_path: str = ''
# project: str='aburdenk-project'
# region: str='us-central1'
# max_template_date: str='2020-05-14',
# models = {'model_name': 'model_1', 'random_seed': 1} 
# #models = list[Mapping]()
# use_gpu_for_relaxation: bool=True
# num_ensemble: int=1
# reference_datasets_uri: str=config.REFERENCE_DATASETS_URI 
# model_params_uri: str=config.MODEL_PARAMS_GCS_LOCATION

api_client = AIPlatformClient(project_id=GOOGLE_CLOUD_PROJECT, region=REGION)
response = api_client.create_run_from_job_spec(
    PIPELINE_SPEC_PATH,
    pipeline_root=PIPELINE_ROOT,
    parameter_values=params
)


# flags.DEFINE_string('pipeline_spec', 'alphafold-inference.json', 'Path to pipeline spec')
# flags.DEFINE_string('pipeline_staging_location', 'gs://alphafold_protein_structure/pipelines', 'Vertex AI staging bucket')
# flags.DEFINE_string('dsub_logging_path', 'gs://alphafold_protein_structure/logging', 'dsub logging')
# flags.DEFINE_string('project', 'aburdenko-project', 'GCP Project')
# flags.DEFINE_string('project_number', '653488387759', 'Project number')
# flags.DEFINE_string('region', 'us-central1', 'GCP Region')
# flags.DEFINE_string('fasta_path', 'gs://alphafold_protein_structure/upload/sequences.fasta', 'A path to a sequence')
# #flags.DEFINE_string('fasta_path', 'gs://jk-alphafold-datasets-archive/fasta/T1050.fasta', 'A path to a sequence')
# flags.DEFINE_string('vertex_sa', 'aburdenko-jupyter-notebook@aburdenko-project.iam.gserviceaccount.com', 'Vertex SA')
# flags.DEFINE_string('pipelines_sa', 'aburdenko-jupyter-notebook@aburdenko-project.iam.gserviceaccount.com', 'Pipelines SA')
# flags.DEFINE_string('uniref90_database_path', 'test1', 'Database paths')
# flags.DEFINE_string('databases_disk_image', 'http://test.com', 'Disk image prepopulated with databases')
# flags.DEFINE_string('max_template_date', '2020-05-14', 'Max template date')
# flags.DEFINE_integer('num_ensemble', 1, 'TBD')
# flags.DEFINE_integer('random_seed', None, 'TBD')
# flags.DEFINE_bool('enable_caching', True, 'Enable pipeline run caching')
# flags.DEFINE_bool('use_gpu_for_relaxation', True, 'Use GPU for relaxation')
# flags.DEFINE_string('sequence_desc', 'T1050 A7LXT1, Bacteroides Ovatus, 779 residues', '')


TypeError: string indices must be integers

Resultant pipeline should look like the following: